# Objective

Interactively experiment with `test_rnn_epoch`, incorporating node labels into graph prediction.

In [1]:
%cd GraphRNN

/home/alpaca/projects/Comp755-Project-GraphGeneration/GraphRNN


In [2]:
from main import *

/home/alpaca/projects/Comp755-Project-GraphGeneration/conda-env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
args = Args()

[graph_type]

[max_num_node]

[max_prev_node]

[hidden_size_rnn_output]

[embedding_size_rnn_output]

[embedding_size_lstm]

[hidden_size]

[batch_size]

[test_batch_size]

[test_total_size]

[num_layers]

[num_workers]

[batch_ratio]

[epochs]

[epochs_test_start]

[epochs_test]

[epochs_log]

[epochs_save]

[lr]

[milestones]

[lr_rate]

[sample_time]

[dir_input]

[load]

[load_epoch]

[save]

[generator_baseline]

[metric_baseline]

Valid input file with  ['[graph_type]\n', '[max_num_node]\n', '[max_prev_node]\n', '[hidden_size_rnn_output]\n', '[embedding_size_rnn_output]\n', '[embedding_size_lstm]\n', '[hidden_size]\n', '[batch_size]\n', '[test_batch_size]\n', '[test_total_size]\n', '[num_layers]\n', '[num_workers]\n', '[batch_ratio]\n', '[epochs]\n', '[epochs_test_start]\n', '[epochs_test]\n', '[epochs_log]\n', '[epochs_save]\n', '[lr]\n', '[milestones]\n', '[lr_rate]\n', '[sample_time]\n', '[dir_input]\n', '[load]\n', '[load_epoch]\n', '[save]\n', '[generator_baseline]\n', '[met

In [4]:
G_list = Graph_load_batch(name='DD', node_attributes=False)

Loading graph dataset: DD
Loaded


In [5]:
G_list = [G for G in G_list if nx.number_connected_components(G) == 1]

In [6]:
dataset = Graph_sequence_sampler_pytorch_nodelabels(G_list, iteration=2000)

calculating max previous node, total iteration: 2000
iter 0 times
iter 400 times
iter 800 times
iter 1200 times
iter 1600 times
max previous node: 216


In [7]:
# need a way to consolidate into "unknown" or "UNK" labels
dataset.num_node_labels

21

In [8]:
dataset.n

903

In [9]:
dataset.node_lab_vocab

{11: 0,
 1: 1,
 8: 2,
 20: 3,
 7: 4,
 12: 5,
 4: 6,
 16: 7,
 10: 8,
 17: 9,
 2: 10,
 15: 11,
 3: 12,
 14: 13,
 6: 14,
 19: 15,
 9: 16,
 13: 17,
 5: 18,
 18: 19,
 65: 20,
 55: 20,
 78: 20,
 76: 20,
 34: 20,
 82: 20,
 58: 20,
 43: 20,
 64: 20,
 31: 20,
 59: 20,
 80: 20,
 56: 20,
 86: 20,
 41: 20,
 37: 20,
 27: 20,
 85: 20,
 88: 20,
 45: 20,
 35: 20,
 54: 20,
 71: 20,
 89: 20,
 25: 20,
 38: 20,
 33: 20,
 70: 20,
 29: 20,
 24: 20,
 39: 20,
 57: 20,
 77: 20,
 81: 20,
 30: 20,
 21: 20,
 69: 20,
 62: 20,
 50: 20,
 68: 20,
 84: 20,
 74: 20,
 66: 20,
 26: 20,
 44: 20,
 46: 20,
 47: 20,
 49: 20,
 75: 20,
 22: 20,
 28: 20,
 67: 20,
 79: 20,
 53: 20,
 36: 20,
 32: 20}

In [10]:
sample_strategy = torch.utils.data.sampler.WeightedRandomSampler([1.0 / len(dataset) for i in range(len(dataset))],
                                                                     num_samples=10, replacement=True)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=10, num_workers=0,
                                               sampler=sample_strategy, drop_last=True)

In [11]:
data = next(iter(data_loader))

In [12]:
data['x'].shape

torch.Size([10, 903, 216])

In [13]:
data['y'].shape

torch.Size([10, 903, 216])

In [14]:
data['labs'].shape

torch.Size([10, 903, 21])

In [15]:
data['len']

tensor([475, 107, 420, 392, 115, 352, 467, 257, 481, 123])

# Graph prediction

Test code in `test_rnn_epoch_nodelabs`, for graph prediction.

In [16]:
rnn = GRU_plain(
    input_size = dataset.max_prev_node, 
    embedding_size = args.embedding_size_rnn,
    hidden_size = args.hidden_size_rnn,
    num_layers = args.num_layers, 
    has_input = True, has_output = True,
    output_size = args.hidden_size_rnn_output
)         

output = GRU_plain(
    input_size = 1,
    embedding_size = args.embedding_size_rnn_output,
    hidden_size = args.hidden_size_rnn_output + args.node_embedding_size,
    num_layers = args.num_layers,
    has_input = True, has_output = True,
    output_size = 1
)

node_embed = MLP_plain(
    h_size = dataset.num_node_labels,
    embedding_size = args.node_embedding_size,
    y_size = args.node_embedding_size
)

node_pred = MLP_plain(
    h_size = args.hidden_size_rnn_output,
    embedding_size = args.node_embedding_size,
    y_size = dataset.num_node_labels
)

/home/alpaca/projects/Comp755-Project-GraphGeneration/GraphRNN/model.py:319: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(param,gain=nn.init.calculate_gain('sigmoid'))
/home/alpaca/projects/Comp755-Project-GraphGeneration/GraphRNN/model.py:317: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(param, 0.25)
/home/alpaca/projects/Comp755-Project-GraphGeneration/GraphRNN/model.py:322: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  m.weight.data = init.xavier_uniform(m.weight.data, gain=nn.init.calculate_gain('relu'))
/home/alpaca/projects/Comp755-Project-GraphGeneration/GraphRNN/model.py:357: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  m.weight.data = init.xavier_uniform(m.weight.data, gain=nn.init.calculate_gain('relu'))


In [17]:
test_batch_size = 16
rnn.hidden = Variable(torch.zeros(args.num_layers, test_batch_size, args.hidden_size_rnn))

In [18]:
max_num_node = 40
num_node_labels = 21
y_pred_long = Variable(torch.zeros(test_batch_size, max_num_node, dataset.max_prev_node))
x_step = Variable(torch.ones(test_batch_size, 1, dataset.max_prev_node))

In [19]:
y_pred_long.shape

torch.Size([16, 40, 216])

In [20]:
x_step.shape

torch.Size([16, 1, 216])

In [21]:
def sample_sigmoid_nocuda(y, sample, thresh=0.5, sample_time=2):
    '''
        do sampling over unnormalized score
    :param y: input
    :param sample: Bool
    :param thresh: if not sample, the threshold
    :param sampe_time: how many times do we sample, if =1, do single sample
    :return: sampled result
    '''

    # do sigmoid first
    y = F.sigmoid(y)
    # do sampling
    if sample:
        if sample_time>1:
            y_result = Variable(torch.rand(y.size(0),y.size(1),y.size(2)))
            # loop over all batches
            for i in range(y_result.size(0)):
                # do 'multi_sample' times sampling
                for j in range(sample_time):
                    y_thresh = Variable(torch.rand(y.size(1), y.size(2)))
                    y_result[i] = torch.gt(y[i], y_thresh).float()
                    if (torch.sum(y_result[i]).data>0).any():
                        break
                    # else:
                    #     print('all zero',j)
        else:
            y_thresh = Variable(torch.rand(y.size(0),y.size(1),y.size(2)))
            y_result = torch.gt(y,y_thresh).float()
    # do max likelihood based on some threshold
    else:
        y_thresh = Variable(torch.ones(y.size(0), y.size(1), y.size(2))*thresh)
        y_result = torch.gt(y, y_thresh).float()
    return y_result

In [22]:
def sample_softmax(y):
    """Select a single class sample from an array of logits."""
    
    y = F.softmax(y, dim = -1)  # softmax on logits for each batch, in last dim
    y_numpy = y.detach().numpy().squeeze()
    # print(y_numpy)
    y_result = Variable(torch.zeros(*y.shape))
    
    # loop over batches
    for i in range(y.size(0)):
        j = np.random.choice(np.arange(y.size(-1)), size = 1, p = y_numpy[i, :])
        y_result[i, :, j] = 1.
    
    return y_result

In [23]:
h = rnn(x_step)

In [24]:
lab_pred = node_pred(h)
lab_pred = sample_softmax(lab_pred)

In [25]:
lab_embedded = node_embed(lab_pred)

In [26]:
hcat = torch.cat((h, lab_embedded), dim = -1)

In [27]:
hidden_null = Variable(torch.zeros(args.num_layers-1, hcat.size(0), hcat.size(2)))

In [28]:
output.hidden = torch.cat((hcat.permute(1,0,2), hidden_null), dim=0)

In [29]:
# now we generate edges for each node predicted so far. 
# Let's pretend that we have generated 10 nodes, so there are 10 edges to predict.

x_step = Variable(torch.zeros(test_batch_size, 1, dataset.max_prev_node))
output_x_step = Variable(torch.ones(test_batch_size, 1, 1))
for j in range(min(dataset.max_prev_node, 10)):
            output_y_pred_step = output(output_x_step)
            output_x_step = sample_sigmoid_nocuda(output_y_pred_step, sample=True, sample_time=1)
            x_step[:,:,j:j+1] = output_x_step
            output.hidden = Variable(output.hidden.data)

/home/alpaca/projects/Comp755-Project-GraphGeneration/conda-env/lib/python3.10/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [30]:
# check to make sure we got edges.
x_step[:, :, 0:9]

tensor([[[1., 0., 0., 0., 0., 1., 1., 0., 0.]],

        [[1., 0., 0., 0., 1., 0., 1., 1., 1.]],

        [[1., 0., 1., 1., 1., 0., 0., 1., 0.]],

        [[0., 1., 0., 0., 0., 1., 1., 0., 0.]],

        [[1., 0., 0., 1., 1., 0., 0., 0., 0.]],

        [[0., 1., 0., 1., 1., 0., 0., 1., 1.]],

        [[0., 1., 1., 1., 0., 1., 1., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 1.]],

        [[0., 0., 0., 0., 1., 0., 0., 0., 0.]],

        [[1., 1., 0., 0., 1., 1., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 1., 0., 1., 1.]],

        [[1., 1., 0., 0., 1., 0., 1., 1., 1.]],

        [[1., 1., 0., 0., 1., 0., 0., 1., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 1., 0., 0., 0., 0.]]])

In [31]:
y_pred_long[:, 0:1, :] = x_step

In [32]:
rnn.hidden = Variable(rnn.hidden.data)

In [33]:
y_pred_long_data = y_pred_long.data.long()

In [34]:
y_pred_long_data

tensor([[[1, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[1, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[1, 0, 1,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[1, 1, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[1, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

### Predict entire graphs

Now run the whole outer for loop:

In [35]:
max_num_node = int(max_num_node)
y_pred_long = Variable(
    torch.zeros(test_batch_size, max_num_node, dataset.max_prev_node)
)# discrete prediction
labs_pred_long = Variable(
    torch.zeros(test_batch_size, max_num_node, dataset.num_node_labels)
)

for i in range(max_num_node):
    h = rnn(x_step)
    lab_pred = node_pred(h)
    lab_pred = sample_softmax(lab_pred)
    lab_embedded = node_embed(lab_pred)
    hcat = torch.cat((h, lab_embedded), dim = -1)

    # output.hidden = h.permute(1,0,2)
    hidden_null = Variable(
        torch.zeros(args.num_layers - 1, hcat.size(0), hcat.size(2))
    )
    output.hidden = torch.cat((hcat.permute(1,0,2), hidden_null), dim=0)  # num_layers, batch_size, hidden_size
    x_step = Variable(torch.zeros(test_batch_size,1,dataset.max_prev_node))
    output_x_step = Variable(torch.ones(test_batch_size,1,1))
    for j in range(min(dataset.max_prev_node,i+1)):
        output_y_pred_step = output(output_x_step)
        output_x_step = sample_sigmoid_nocuda(output_y_pred_step, sample=True, sample_time=1)
        x_step[:,:,j:j+1] = output_x_step
        output.hidden = Variable(output.hidden.data)
    y_pred_long[:, i:i + 1, :] = x_step
    labs_pred_long[:, i:i+1, :] = lab_pred
    rnn.hidden = Variable(rnn.hidden.data)
y_pred_long_data = y_pred_long.data.long()
labs_pred_long_data = labs_pred_long.data.long()

In [36]:
y_pred_long_data.shape

torch.Size([16, 40, 216])

In [37]:
labs_pred_long_data.shape

torch.Size([16, 40, 21])

In [38]:
y_pred_long_data[:, -1, :]

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0]])

It works!!

### Saving graphs

In [46]:
G_pred_list = []
for i in range(test_batch_size):
    adj_pred = decode_adj(y_pred_long_data[i].detach().numpy())
    labs_decoded = decode_node_labels(labs_pred_long_data[i].detach().numpy(), dataset.reverse_vocabulary())
    G_pred = get_graph(adj_pred)
    G_pred.remove_node(0)  # remove first node since it does not have a label
    for node in G_pred.nodes:
        G_pred.nodes[node]['label'] = labs_decoded[node-1]
    G_pred_list.append(G_pred)